## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/Resid6_NeuralNetworks/LabInternal/prices.csv")



### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis =1,inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_df = data.iloc[0:1000]

In [11]:
data_df.shape

(1000, 5)

In [12]:
data_df['volume'] = data_df['volume'] / 1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
y = data_df['volume']
x = data_df.drop(['volume'], axis = 1)

In [15]:
x.shape, y.shape

((1000, 4), (1000,))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.3, random_state =5)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [0]:
train_y=train_y.reshape(-1,1)

In [20]:
train_y.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,2))
b1 = tf.zeros(shape=(2)) 


In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(2,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        #current_loss =tf.nn.sigmoid_cross_entropy_with_logits(labels = train_y,logits =net2 )
        current_loss = loss(net2, train_y)
        print(current_loss)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    #print("Loss at step {:d}: {:.3f}".format(i, currentloss)
    
    return w1, b1,w2,b2


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [27]:
for i in range(100):
    
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

tf.Tensor(255.76656, shape=(), dtype=float32)
tf.Tensor(254.59814, shape=(), dtype=float32)
tf.Tensor(253.47589, shape=(), dtype=float32)
tf.Tensor(252.3978, shape=(), dtype=float32)
tf.Tensor(251.36201, shape=(), dtype=float32)
tf.Tensor(250.36688, shape=(), dtype=float32)
tf.Tensor(249.41075, shape=(), dtype=float32)
tf.Tensor(248.49232, shape=(), dtype=float32)
tf.Tensor(247.6101, shape=(), dtype=float32)
tf.Tensor(246.76286, shape=(), dtype=float32)
tf.Tensor(245.94957, shape=(), dtype=float32)
tf.Tensor(245.16913, shape=(), dtype=float32)
tf.Tensor(244.42052, shape=(), dtype=float32)
tf.Tensor(243.70293, shape=(), dtype=float32)
tf.Tensor(243.01543, shape=(), dtype=float32)
tf.Tensor(242.35724, shape=(), dtype=float32)
tf.Tensor(241.72766, shape=(), dtype=float32)
tf.Tensor(241.1259, shape=(), dtype=float32)
tf.Tensor(240.55125, shape=(), dtype=float32)
tf.Tensor(240.00304, shape=(), dtype=float32)
tf.Tensor(239.48058, shape=(), dtype=float32)
tf.Tensor(238.98323, shape=(), dtype=

In [28]:
for i in range(0,100,5):
    print(i)
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

0
tf.Tensor(231.23598, shape=(), dtype=float32)
5
tf.Tensor(231.23451, shape=(), dtype=float32)
10
tf.Tensor(231.23299, shape=(), dtype=float32)
15
tf.Tensor(231.23169, shape=(), dtype=float32)
20
tf.Tensor(231.23045, shape=(), dtype=float32)
25
tf.Tensor(231.22928, shape=(), dtype=float32)
30
tf.Tensor(231.2283, shape=(), dtype=float32)
35
tf.Tensor(231.2273, shape=(), dtype=float32)
40
tf.Tensor(231.22638, shape=(), dtype=float32)
45
tf.Tensor(231.22563, shape=(), dtype=float32)
50
tf.Tensor(231.22484, shape=(), dtype=float32)
55
tf.Tensor(231.22415, shape=(), dtype=float32)
60
tf.Tensor(231.22342, shape=(), dtype=float32)
65
tf.Tensor(231.22284, shape=(), dtype=float32)
70
tf.Tensor(231.2223, shape=(), dtype=float32)
75
tf.Tensor(231.22173, shape=(), dtype=float32)
80
tf.Tensor(231.2213, shape=(), dtype=float32)
85
tf.Tensor(231.22092, shape=(), dtype=float32)
90
tf.Tensor(231.22047, shape=(), dtype=float32)
95
tf.Tensor(231.22005, shape=(), dtype=float32)


### Get the shapes and values of W and b

In [29]:
w1

<tf.Tensor: id=6495, shape=(4, 2), dtype=float32, numpy=
array([[-0.04754365, -1.6142803 ],
       [ 0.7270954 , -1.23668   ],
       [ 0.82412225, -1.0792744 ],
       [-0.6058177 , -0.20622554]], dtype=float32)>

In [30]:
w1.shape,w2.shape

(TensorShape([Dimension(4), Dimension(2)]),
 TensorShape([Dimension(2), Dimension(1)]))

In [31]:
w2

<tf.Tensor: id=6489, shape=(2, 1), dtype=float32, numpy=
array([[ 2.384499  ],
       [-0.12639166]], dtype=float32)>

In [32]:
b1

<tf.Tensor: id=6498, shape=(2,), dtype=float32, numpy=array([ 0.95163095, -0.09629563], dtype=float32)>

In [33]:
b2

<tf.Tensor: id=6492, shape=(1,), dtype=float32, numpy=array([3.162468], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [0]:
y_pred,current_prediction = prediction(test_x, w1, b1,w2,b2)

In [40]:
y_pred

<tf.Tensor: id=6518, shape=(300, 1), dtype=float32, numpy=
array([[5.060862 ],
       [5.059683 ],
       [5.060196 ],
       [5.0599804],
       [5.0596924],
       [5.0610623],
       [5.059669 ],
       [5.052343 ],
       [5.0592055],
       [5.0609055],
       [5.0614595],
       [5.060857 ],
       [5.059287 ],
       [5.059361 ],
       [5.0606313],
       [5.057459 ],
       [5.0598683],
       [5.060356 ],
       [5.05824  ],
       [5.0610476],
       [5.057955 ],
       [5.060362 ],
       [5.060951 ],
       [5.060331 ],
       [5.0550494],
       [5.059725 ],
       [5.0587068],
       [5.059736 ],
       [5.060684 ],
       [5.060588 ],
       [5.059203 ],
       [5.060536 ],
       [5.060354 ],
       [5.059995 ],
       [5.061109 ],
       [5.0615034],
       [5.06066  ],
       [5.0593824],
       [5.0610933],
       [5.0607567],
       [5.0609093],
       [5.06099  ],
       [5.061557 ],
       [5.060201 ],
       [5.060755 ],
       [5.0611367],
       [5.054666 ],
 

In [44]:
from sklearn.metrics import r2_score
print(r2_score(test_y,y_pred))

-0.003112086616862575


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/Resid6_NeuralNetworks/LabInternal/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [48]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
y = pd.get_dummies(iris['Species'])

### Splitting the data into feature set and target set

In [56]:
x = iris.iloc[:,1:5]
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [57]:
y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [60]:
x.shape

(150, 4)

In [71]:
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
tf.disable_eager_execution()

In [0]:
model = Sequential()
model.add(Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal', activation='softmax'))
# Compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.3, random_state =2)

### Model Training 

In [105]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 2ms/step - loss: 1.0935 - acc: 0.3524 - val_loss: 1.0932 - val_acc: 0.2889
Epoch 2/50
105/105 [==============================] - 0s 76us/step - loss: 1.0912 - acc: 0.3524 - val_loss: 1.0912 - val_acc: 0.2889
Epoch 3/50
105/105 [==============================] - 0s 80us/step - loss: 1.0888 - acc: 0.3524 - val_loss: 1.0891 - val_acc: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 72us/step - loss: 1.0862 - acc: 0.3524 - val_loss: 1.0880 - val_acc: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 73us/step - loss: 1.0847 - acc: 0.3524 - val_loss: 1.0871 - val_acc: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 75us/step - loss: 1.0831 - acc: 0.3524 - val_loss: 1.0861 - val_acc: 0.2889
Epoch 7/50
105/105 [==============================] - 0s 73us/step - loss: 1.0817 - acc: 0.3524 - val_loss: 1.0851 - val_acc: 0.2889
Epoch 8/50
105/105 [=====

### Model Prediction

In [0]:
y_pred = model.predict(test_x)

In [107]:
y_pred

array([[0.3543337 , 0.3341378 , 0.31152844],
       [0.345372  , 0.33660108, 0.31802693],
       [0.1512675 , 0.35948396, 0.4892485 ],
       [0.35105348, 0.33500454, 0.313942  ],
       [0.33183756, 0.34017304, 0.32798934],
       [0.14079489, 0.3583761 , 0.50082904],
       [0.33807698, 0.3384774 , 0.32344562],
       [0.13185188, 0.35643065, 0.51171756],
       [0.13628949, 0.35735112, 0.5063594 ],
       [0.3453855 , 0.33654547, 0.31806907],
       [0.35679203, 0.33335084, 0.30985713],
       [0.3566358 , 0.33351225, 0.3098519 ],
       [0.3567875 , 0.33335555, 0.30985698],
       [0.34727728, 0.3359887 , 0.31673405],
       [0.18720298, 0.3619327 , 0.45086434],
       [0.17668858, 0.36161622, 0.46169516],
       [0.33500797, 0.33930525, 0.32568675],
       [0.19604144, 0.3617579 , 0.4422007 ],
       [0.13345282, 0.3569236 , 0.5096235 ],
       [0.19697137, 0.3616295 , 0.44139916],
       [0.162934  , 0.36092505, 0.47614092],
       [0.19240348, 0.36172536, 0.44587117],
       [0.

In [108]:
#y_test_class = np.argmax(test_y,axis = 1)
y_pred_class = np.argmax(y_pred, axis = 1)
y_pred_class

array([0, 0, 2, 0, 1, 2, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       0])

In [120]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 127us/step


[0.939855408668518, 0.5555555562178294]

### Save the Model

In [0]:
model.save('Iris_model.html')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?